# Clasificación de iris con red neuronal simple

In [11]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
 
iris = load_iris()
print(iris.data.dtype)
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)
 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

float64


Convertimos los arrays de NumPy a tensores de PyTorch.

Scikit-learn utiliza a menudo floats de 64 bits para tener más precisión, pero PyTorch utiliza floats de 32 bits por defecto, priorizando la eficiencia al ser utilizado para datasets mas grandes normalmente más robustosa pequeñas variaciones en los datos.

Por lo tanto, convertimos los arrays de numpy a tensores de PyTorch pasando el tipo de dato a `torch.float32`.

In [12]:
import torch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train)

Definimos un modelo de red neuronal simple parametrizable con el número de entradas y salidas y el número de neuronas en la capa oculta.

In [13]:
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Input layer
        self.relu = nn.ReLU()                          # Activation function
        self.fc2 = nn.Linear(hidden_size, output_size) # Output layer
         
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x    

Intanciamos la red neuronal con el número de entradas y salidas correspondientes a los datos de iris, y 10 neuronas en la capa oculta.

Definimos el optimizador Adam y la función de pérdida de entropía cruzada.

In [14]:
torch.manual_seed(42) # Fijamos la semilla para asegurar reproducibilidad

input_size = iris.data.shape[1] # Capa de entrada: número de características
hidden_size = 10 # Tamaño de la capa oculta
output_size = len(iris.target_names) # Capa de salida: número de clases

model = SimpleNN(input_size, hidden_size, output_size) # Instanciamos el modelo


criterion = nn.CrossEntropyLoss() # Función de pérdida
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.01) # Optimizador

Realizamos el entrenamiento del modelo. En este aso, al ser un dataset muy pequeño, pasamos todos los datos de entrenamiento en un solo ***batch***, por lo que no se realiza un entrenamiento por lotes.

In [15]:
num_epochs = 1000
for epoch in range(num_epochs):
    outputs = model(X_train_tensor) # Forward pass
    loss = criterion(outputs, y_train_tensor) # Calcular la pérdida
    
    optimizer.zero_grad() # Inicializar los gradientes a cero
    loss.backward() # Backward pass
    optimizer.step() # Actualizar los parámetros
    
    if (epoch+1) % 10 == 0: # Cada 10 epochs muestra la pérdida
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}') 

Epoch [10/1000], Loss: 0.7783
Epoch [20/1000], Loss: 0.5399
Epoch [30/1000], Loss: 0.3921
Epoch [40/1000], Loss: 0.2934
Epoch [50/1000], Loss: 0.2166
Epoch [60/1000], Loss: 0.1639
Epoch [70/1000], Loss: 0.1284
Epoch [80/1000], Loss: 0.1050
Epoch [90/1000], Loss: 0.0902
Epoch [100/1000], Loss: 0.0800
Epoch [110/1000], Loss: 0.0732
Epoch [120/1000], Loss: 0.0683
Epoch [130/1000], Loss: 0.0647
Epoch [140/1000], Loss: 0.0619
Epoch [150/1000], Loss: 0.0597
Epoch [160/1000], Loss: 0.0579
Epoch [170/1000], Loss: 0.0564
Epoch [180/1000], Loss: 0.0551
Epoch [190/1000], Loss: 0.0541
Epoch [200/1000], Loss: 0.0532
Epoch [210/1000], Loss: 0.0524
Epoch [220/1000], Loss: 0.0517
Epoch [230/1000], Loss: 0.0511
Epoch [240/1000], Loss: 0.0506
Epoch [250/1000], Loss: 0.0501
Epoch [260/1000], Loss: 0.0498
Epoch [270/1000], Loss: 0.0494
Epoch [280/1000], Loss: 0.0491
Epoch [290/1000], Loss: 0.0488
Epoch [300/1000], Loss: 0.0486
Epoch [310/1000], Loss: 0.0484
Epoch [320/1000], Loss: 0.0482
Epoch [330/1000],

Comprobamos la exactitud del modelo con el conjunto de test.

In [16]:
with torch.no_grad():
	X_test_tensor = torch.FloatTensor(X_test)
	y_test_tensor = torch.LongTensor(y_test)
	outputs = model(X_test_tensor)
	_, predicted = torch.max(outputs, 1)
	accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
	print(f'Accuracy on the test set: {accuracy:.2f}')

Accuracy on the test set: 1.00
